In [ ]:
import sys
import os
import re
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


fields = [ 'qname', 'hostname', 'group', 'owner', 'job_name',
      'job_number', 'account', 'priority', 'submission_time', 'start_time',
      'end_time', 'failed', 'exit_status', 'ru_wallclock', 'ru_utime',
      'ru_stime', 'ru_maxrss', 'ru_ixrss', 'ru_ismrss', 'ru_idrss',
      'ru_isrss', 'ru_minflt', 'ru_majflt', 'ru_nswap', 'ru_inblock',
      'ru_oublock', 'ru_msgsnd', 'ru_msgrcv', 'ru_nsignals', 'ru_nvcsw',
      'ru_nivcsw', 'project', 'department', 'granted_pe', 'slots',
      'task_number', 'cpu', 'mem', 'io', 'category',
      'iow', 'pe_taskid', 'maxvmem', 'arid', 'ar_submission_time' ]

usecols = ['owner', 'group', 'job_number', 'task_number','granted_pe',
        'slots' ,'category','start_time', 'end_time','failed', 'exit_status','submission_time']

df=pd.DataFrame( ['owner', 'group', 'job_number', 'task_number','granted_pe',
        'slots' ,'category','start_time', 'end_time','failed', 'exit_status'])

data=pd.read_csv('accounting-2018-10.dms', sep=':', error_bad_lines=False, header=None,
        names=fields, usecols=usecols, encoding = "ISO-8859-1")

df=pd.DataFrame(data,columns=['owner', 'group', 'job_number', 'task_number','granted_pe',
        'slots' ,'category','start_time', 'end_time','failed', 'exit_status','submission_time'])

df.head()


# Step 4-1
A typical category field looks like:

...:-U campus -u stevendu -l h_data=4G,h_rt=86400,h_vmem=4G -pe single 1:...
In the category field, extract the h_data data, covert the value to gigabytes (see below for explanation) and make it a new column.

If the value of h_data ends with a “G” or “g”, the data is in the unit of “gigabytes”. If the value ends with “m” or “M”, the data is in the unit of megabytes:

20M or 20m  : 20 megabytes
4G or 4g    : 4 gigabytes
1024        : 1024 bytes
For example, if the category field has h_data=2048M,h_rt=86400,exclusive=TRUE, extract the 2048M, and convert it to 2048 / 1024 = 2 (gigabytes). (Recall: 1G = 1024M).

In [ ]:
df['h_data']=df['category'].str.extract("h_data=(.*.),h_rt", expand=True)

updt = df['h_data']
for i in range (0,len(df['h_data'])):
    try:
        if (updt[i].find('M') != -1.0) or updt[i].find('m') != -1.0 :
            updt[i] = updt[i][:-1]
            updt[i] = round(float(updt[i]) / 1024,2)
            updt[i] = str(updt[i]) + 'G'
    except:
        continue 

df



/Volumes/Everest/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Volumes/Everest/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Volumes/Everest/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Step 4-2
In the category field, extract the h_rt data, which is in seconds. Make a new column for h_rt in the unit of hours. For example, if the h_rt value is 86400, convert it to 86400/(3600*24) = 24 (hours). In this case, the row value in the new h_rt column will be 24.

In [ ]:
df['h_rt']=df['category'].str.extract("h_rt=(.*.),h_vmem", expand=True)
df["h_rt"]=df['h_rt'].astype('float')
df["h_rt"]=df[('h_rt')]/(3600)
df


# Step 4-3
Create a new column called highp. In the category field, if highp=TRUE or highp=true is identified, the row value of highp would be 1. Otherwise highp is 0 in the new column.

In [ ]:
df['highp']=df['category'].str.extract("highp=(.*.), -pe")
df["highp"]=(df['highp']=='TRUE').astype(int) + (df['highp']=='true').astype(int)


# Step 4-4
Create a new column called exclusive. In the category field, if exclusive=TRUE or exclusive=true is identified, the row value of exclusive would be 1. Otherwise exclusive is 0 in the new column.

In [ ]:
df['exclusive']=df['category'].str.extract("exclusive=(.*.),h_data")
df["exclusive"]=(df['exclusive']=='TRUE').astype(int) + (df['exclusive']=='true').astype(int)

# Step 4-5
Create a new column called h_vmem. Look for its value in the category field. Similar to h_data, convert the values to gigabytes.

In [ ]:
df['h_vmem']=df['category'].str.extract("h_vmem=(.*.) -pe", expand=True)

updt = df['h_vmem']
for i in range (0,len(df['h_vmem'])):
     try:
            if updt[i].find('INF') != -1.0:
                 updt[i]= 0
                  
            if (updt[i].find('M') != -1.0) or updt[i].find('m') != -1.0 :
                updt[i] = updt[i][:-1]
                updt[i] = round(float(df2[i]) / 1024,2)
                updt[i] = str(updt[i]) + 'G'
     except:
        continue 
df


# Step 4-6
Create a new column called gpu. Look for the value in the category field. If required_gpu is identified, set the row value to 1. Otherwise the row value is 0.

In [ ]:
df['gpu'] = df['category'].str.contains('gpu')
df['gpu'] = df['gpu'].map({True: '1', False: '0'})
df


# Step 4-7
Create two new columns. One is named pe. Another one is slot. In the category field, look for the -pe data, e.g. -pe single 1. In this case, put the single (string) in the new pe column, and the value 1 (integer) in the new slot column.

If no pe data is found in the category field, enter none for the pe column, and 1 for the slot column.

In [ ]:
df['pe']=df['category'].str.extract("-pe (.*.) (\d)")
df


In [ ]:
df['slot'] = df['category'].str[-1:]


# Step 4-8
Create a new column campus. In the category field, if the value following -U is campus, set the value to 1 (integer). Otherwise, set it to 0.

In [ ]:
df['campus']=df['category'].str.extract("-U (.*.) -u")
df["campus"]=(df['campus']=='campus').astype(int)
df

# Step 5
The raw data in the start_time, end_time and submission_time are the UNIX epoch time. Convert the data strings to a Pandas (or Python) data objects. The Timestamp function in Pandas can do this easily. See its documentation for details.

In [ ]:
df["submission_time"]=pd.to_datetime(df['submission_time'], unit='s') 
df["start_time"]=pd.to_datetime(df['start_time'], unit='s') 
df["end_time"]=pd.to_datetime(df['end_time'], unit='s') 

# Step 6
Create a new column wait_time whose value is the difference of start_time - submission_time.

Create a new column wtime (short for “wall-clock time”) whose value is the difference of end_time - start_time.

In [ ]:
df["wait_time"]=df['start_time']-df['submission_time']
df["wtime"]=df['end_time']-df['start_time']
df

# Step 7
Identify duplicates and merge them. You can use the Pandas drop_duplicates function.

In [ ]:
df.drop_duplicates(subset=['job_number', 'task_number', 'submission_time'], keep='first', inplace=True)
df.shape

# Step 8
You may or may not find some values in submission_time, start_time or end_time contain dates before year 2018. Drop (delete) these rows from the data.

In [ ]:
df[(df['submission_time'].dt.year < 2018)]
df[(df['start_time'].dt.year < 2018)]
df[(df['end_time'].dt.year < 2018)]



# Step 9
Remove these columns that we will not need: ['category', 'qname', 'job_name', 'account', 'project']

In [ ]:
df.drop(['category'], axis=1, inplace=True)
df.shape


In [ ]:
# pd.set_option('display.max_colwidth', -1)
# print(df['category'])

In [ ]:
df